In [1]:
import os

os.environ["KERAS_BACKEND"] = "jax"

In [2]:
# import py7zr
import time

import keras_nlp
import keras
import tensorflow as tf
from utils import remove_splchars
from pyarrow.parquet import ParquetFile
import multiprocessing
import pyarrow as pa
import numpy as np

from simpletransformers.seq2seq import Seq2SeqModel,Seq2SeqArgs
# import tensorflow_datasets as tfds

2024-03-31 16:56:07.215389: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-31 16:56:08.480935: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/mohitavva/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
BATCH_SIZE = 8
NUM_BATCHES = 600
EPOCHS = 1  # Can be set to a higher value for better results
MAX_ENCODER_SEQUENCE_LENGTH = 512
MAX_DECODER_SEQUENCE_LENGTH = 128
MAX_GENERATION_LENGTH = 40

In [4]:
def get_data():
    data = ParquetFile("./parquet/train.parquet")
    batch = next(data.iter_batches(batch_size = 1000)) 
    batch = pa.Table.from_batches([batch]).to_pandas()
    batch.drop(["id"], axis=1, inplace=True)

    #pool = multiprocessing.Pool()
    with multiprocessing.Pool(5) as p:
        batch["article"] = p.map(remove_splchars,batch["article"])
    
    
    return batch

In [5]:
data = get_data()
data.columns = ['input_text', 'target_text']



def load_ds(data, batch_size):
    values = data.values
    print(values.shape)
    batched = tf.split(values, batch_size, axis=0)

    
    l = [{"encoder_text": (i[:, 0]), "decoder_text": i[:, 1]} for i in batched]
    return l

train_ds = load_ds(data, 125)
# train_ds = train_ds.take(NUM_BATCHES)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


(1000, 2)


2024-03-31 16:56:17.722162: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-31 16:56:17.726893: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-31 16:56:17.726964: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-31 16:56:17.728473: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-31 16:56:17.728554: I external/local_xla/xla/stream_executor

In [6]:
model_args = Seq2SeqArgs()
model_args.num_train_epochs = 10
model_args.no_save = True
model_args.evaluate_generated_text = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = True

# Initialize model
model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="facebook/bart-base",
    args=model_args,
    use_cuda=True,
)
    

In [7]:
model.train_model(data, eval_data=data)

  0%|          | 0/1000 [00:05<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
print(model.predict(
        [
            "Tyson is a Cyclops, a son of Poseidon, and Percy Jackson’s half brother. He is the current general of the Cyclopes army."
        ]
    )
)